In [1]:
from __future__ import print_function
from keras.models import load_model
import numpy as np
import sklearn
import sklearn.model_selection
from utils import encode, get_save_callback
from models import build_model

Using TensorFlow backend.


In [2]:
with open("data\\full_text.txt", "r", encoding="utf8") as f:
    full_text = f.read()

In [3]:
labels = list(np.load("data\\labels.npy"))

In [4]:
SEQUENCE_LENGTH = 10
NUM_INPUTS = len(labels)
TEST_SPLIT_PERCENTAGE=0.2
VAL_SPLIT_PERCENTAGE=0.2
MODEL = "M5"
model_id = 13

In [5]:
model_id += 1
model = build_model(MODEL, SEQUENCE_LENGTH, NUM_INPUTS)

In [6]:
training_text = full_text[2000000:]

In [7]:
x_shape = (len(training_text)//2000//3, SEQUENCE_LENGTH-1, len(labels))
i = 1
for j in x_shape:
    i *= j
print(i)
x = np.zeros(x_shape, dtype=np.bool)
y = np.zeros((x_shape[0], len(labels)), dtype=np.bool)

for rid, i in enumerate(range(0, len(training_text)//2000-3, 3)):
    for j in range(SEQUENCE_LENGTH-1):
        character = training_text[i+j]
        index = encode(character, labels)
        x[rid, j, index] = 1
    index = encode(training_text[i+j+1], labels)
    y[rid, index] = 1
    if i % 2000 == 0:
        print(i)

1212191532
0
6000
12000
18000
24000
30000
36000
42000
48000
54000
60000
66000
72000
78000
84000
90000
96000
102000
108000
114000
120000
126000
132000
138000
144000
150000


In [8]:
x, y = sklearn.utils.shuffle(x, y, random_state=0)

In [9]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=TEST_SPLIT_PERCENTAGE, random_state=42)

In [ ]:
del x
del y

In [ ]:
model.fit(x_train, y_train, batch_size=256, epochs=3, callbacks=[get_save_callback(model_id)], validation_split=VAL_SPLIT_PERCENTAGE)

Train on 32888 samples, validate on 8222 samples
Epoch 1/3
32888/32888 [==============================] - 50s - loss: 5.8300 - acc: 0.0561 - val_loss: 5.3910 - val_acc: 0.0937
Epoch 2/3
32888/32888 [==============================] - 51s - loss: 4.8983 - acc: 0.1560 - val_loss: 4.7422 - val_acc: 0.1880
Epoch 3/3
20736/32888 [=================>............] - ETA: 17s - loss: 4.3084 - acc: 0.2266

In [ ]:
del x_train, y_train, x_test, y_test